# Find Optimal PSF
===========================

- creation 06/07/2016
- author Sylvie Dagoret-Campagne



Find HD163466 with Y4

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from astropy.modeling import models
from astropy import units as u
from astropy import nddata
from astropy.io import fits

from astropy.table import Table
from astropy.table import Column

import ccdproc
print 'ccdproc version',ccdproc.__version__

from astropy.modeling import models

ccdproc version 1.0.1


In [2]:
import photutils
from astropy.stats import sigma_clipped_stats
from photutils import daofind
from photutils import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

In [3]:
from scipy import stats 
from scipy import ndimage
import os
from datetime import datetime, timedelta

In [4]:
from photutils.background import Background2D

In [5]:
import libMonocamBaseImages           # my tool library written to do that CCD reduction

In [6]:
now=datetime.utcnow()  # choose UTC time
datestr=str(now)
print 'standard date format for the analysis :',datestr
#  want the following format '2016-05-10T11:55:27.267'
date_of_analysis=now.strftime('%Y-%m-%dT%H:%M:%S')
print 'fits date format for the analysis : ',date_of_analysis

standard date format for the analysis : 2016-07-06 20:35:30.012361
fits date format for the analysis :  2016-07-06T20:35:30


## Definitions and Constants

In [7]:
object_name='HD163466_y4'

### input files

In [8]:
path='./HD163466_y4'
rootfilename='AssScImHD163466_y4_' 


NumStart=1
NumStop=8

### output file (table)

In [9]:
outputtablefile='HD163466_y4_1-8_TablePSF.fits'

### make the filelist

In [10]:
filelist=libMonocamBaseImages.BuildFilelist(path,rootfilename,start=NumStart,stop=NumStop,nbchar=1)

In [11]:
filelist

['./HD163466_y4/AssScImHD163466_y4_1.fits',
 './HD163466_y4/AssScImHD163466_y4_2.fits',
 './HD163466_y4/AssScImHD163466_y4_3.fits',
 './HD163466_y4/AssScImHD163466_y4_4.fits',
 './HD163466_y4/AssScImHD163466_y4_5.fits',
 './HD163466_y4/AssScImHD163466_y4_6.fits',
 './HD163466_y4/AssScImHD163466_y4_7.fits',
 './HD163466_y4/AssScImHD163466_y4_8.fits']

## Read Input files

In [12]:
allchannelallsciimages = []  # list of 16 lists of images series 
exposures_list = []        # sequential list of the exposures of the sky flats 
header_list = []           # list of headers
data_list = []
time_list = []                # date and time
basefile_list = []         # basefilename
dateobs_list = [] 
# get the primary block headers:
for image_file in filelist: 
    print image_file
    hdu_list = fits.open(image_file)
    basefile_list.append(os.path.basename(image_file))
    header=hdu_list[0].header
    exposure=header['EXPOSURE']
    exposures_list.append(exposure)
    dateobs_list.append(header['DATE-OBS'])
    header_list.append(header)
    data=ccdproc.CCDData.read(image_file, hdu=0,unit='adu') 
    data_list.append(data)

./HD163466_y4/AssScImHD163466_y4_1.fits
./HD163466_y4/AssScImHD163466_y4_2.fits
./HD163466_y4/AssScImHD163466_y4_3.fits
./HD163466_y4/AssScImHD163466_y4_4.fits
./HD163466_y4/AssScImHD163466_y4_5.fits
./HD163466_y4/AssScImHD163466_y4_6.fits
./HD163466_y4/AssScImHD163466_y4_7.fits
./HD163466_y4/AssScImHD163466_y4_8.fits


In [13]:
#basefile_list

## For control

uncomment for control

In [14]:
index=0

In [15]:
#print exposures_list[index]

In [16]:
#header_list[index]

In [17]:
#plt.imshow(data_list[index])

In [18]:
#bkg= Background2D(data_list[index], (100, 100), filter_size=(3, 3),method='median')

In [19]:
#data_list[index].data-bkg.background

## Background subtraction

In [20]:
correctedimage_list = []

In [21]:
for data in data_list:
    bkg= Background2D(data, (100, 100), filter_size=(3, 3),method='median')
    newimage=data-bkg.background
    correctedimage_list.append(newimage)

## Calculation of PSF

In [22]:
NBIMAGES=len(correctedimage_list)

In [23]:
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return (average, np.sqrt(variance))

In [24]:
image_psf=np.zeros((NBIMAGES,6))

In [25]:
DELTA_NBINSX=10
DELTA_NBINSY=10

In [26]:
# loop on images
index=-1
for image in correctedimage_list:
    index=index+1
    mean, median, std = sigma_clipped_stats(image, sigma=10.0, iters=5) 
    sources = daofind(image - median, fwhm=3.0, threshold=50.*std)
    selected_stars=sources.as_array()
    NBSTARS=selected_stars.shape[0]
    print 'image {} ==> NBSTARS = {}'.format(index,NBSTARS)
    image_psf[index,0]=index
    image_psf[index,1]=NBSTARS
    star_psfx=np.zeros(NBSTARS)
    star_psfy=np.zeros(NBSTARS)
    # loop on stars
    for istar in range(NBSTARS):
        X = int(selected_stars[istar][1])
        Y = int(selected_stars[istar][2])
        prf_image = image[Y-DELTA_NBINSY:Y+DELTA_NBINSY,X-DELTA_NBINSX:X+DELTA_NBINSX]
        oneprfX=prf_image.sum(axis=0)
        oneprfY=prf_image.sum(axis=1)
        if oneprfX.sum() == 0 or oneprfY.sum() == 0:
            star_psfx[istar]=0
            star_psfy[istar]=0
        else:
            posX,sigX=weighted_avg_and_std(np.arange(oneprfX.shape[0]),oneprfX)
            posY,sigY=weighted_avg_and_std(np.arange(oneprfY.shape[0]),oneprfY)
            star_psfx[istar]=sigX
            star_psfy[istar]=sigY
        
    all_sigx=star_psfx[np.logical_not(np.isnan(star_psfx))]
    all_sigy=star_psfy[np.logical_not(np.isnan(star_psfy))]
    all_sigx=all_sigx[all_sigx>2.4]
    all_sigy=all_sigy[all_sigy>2.4]
    print 'average prf(x) = {:2.2f} +/- {:2.2f} pixels ==> psf  {:2.2f} +/- {:2.2f} arcsec '.format(np.median(all_sigx),all_sigx.std(),np.median(all_sigx)*0.4*2.36,all_sigx.std()*0.4*2.26)
    print 'average prf(y) = {:2.2f} +/- {:2.2f} pixels ==> psf  {:2.2f} +/- {:2.2f} arcsec '.format(np.median(all_sigy),all_sigy.std(),np.median(all_sigy)*0.4*2.36,all_sigy.std()*0.4*2.26)
    image_psf[index,2]=np.median(all_sigx)
    image_psf[index,3]=np.median(all_sigy)
    image_psf[index,4]=all_sigx.std()
    image_psf[index,5]=all_sigy.std()

image 0 ==> NBSTARS = 23
average prf(x) = 2.81 +/- 0.07 pixels ==> psf  2.65 +/- 0.06 arcsec 
average prf(y) = 2.94 +/- 0.08 pixels ==> psf  2.77 +/- 0.07 arcsec 
image 1 ==> NBSTARS = 23
average prf(x) = 2.69 +/- 0.08 pixels ==> psf  2.54 +/- 0.07 arcsec 
average prf(y) = 2.87 +/- 0.05 pixels ==> psf  2.71 +/- 0.04 arcsec 
image 2 ==> NBSTARS = 25
average prf(x) = 2.64 +/- 0.08 pixels ==> psf  2.50 +/- 0.07 arcsec 
average prf(y) = 2.69 +/- 0.03 pixels ==> psf  2.54 +/- 0.03 arcsec 


/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in sqrt


image 3 ==> NBSTARS = 30
average prf(x) = 2.67 +/- 0.07 pixels ==> psf  2.52 +/- 0.06 arcsec 
average prf(y) = 2.73 +/- 0.25 pixels ==> psf  2.58 +/- 0.23 arcsec 
image 4 ==> NBSTARS = 16
average prf(x) = 2.86 +/- 0.24 pixels ==> psf  2.70 +/- 0.21 arcsec 
average prf(y) = 2.77 +/- 0.07 pixels ==> psf  2.61 +/- 0.07 arcsec 
image 5 ==> NBSTARS = 14
average prf(x) = 2.79 +/- 0.08 pixels ==> psf  2.63 +/- 0.07 arcsec 
average prf(y) = 2.88 +/- 0.06 pixels ==> psf  2.71 +/- 0.05 arcsec 
image 6 ==> NBSTARS = 11
average prf(x) = 2.78 +/- 0.06 pixels ==> psf  2.62 +/- 0.06 arcsec 
average prf(y) = 2.90 +/- 0.04 pixels ==> psf  2.74 +/- 0.04 arcsec 
image 7 ==> NBSTARS = 13
average prf(x) = 2.66 +/- 0.07 pixels ==> psf  2.51 +/- 0.07 arcsec 
average prf(y) = 2.80 +/- 0.08 pixels ==> psf  2.65 +/- 0.07 arcsec 


## Create astropy table

In [27]:
t=Table(rows=image_psf,names=('num','nbstars','prfx','pfry','sig_prfx','sig_prfy'),dtype=('i4','i4','f8','f8','f8','f8'))

In [28]:
t

num,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,int32,float64,float64,float64,float64
0,23,2.80810530353,2.93709578691,0.0717107607457,0.0760826986652
1,23,2.68655532696,2.86879575208,0.0819848604541,0.0456488029924
2,25,2.64452856755,2.69076001366,0.0755942751507,0.0286223131641
3,30,2.6723065947,2.72932681468,0.0708837703989,0.249819800909
4,16,2.85770605392,2.76551370117,0.236654258462,0.0724385881085
5,14,2.78575614456,2.87539302862,0.0807985674717,0.0585594161166
6,11,2.77968098699,2.90044545098,0.0623518794852,0.0405807569436
7,13,2.66223236329,2.8031331225,0.0747038608423,0.08107409864


In [29]:
expo = Column(exposures_list, name='exposure')
file = Column(basefile_list, name='file')
time = Column(dateobs_list,name='time')

In [30]:
t.add_column(expo, index=1)
t.add_column(time, index=1)
t.add_column(file, index=1)

In [31]:
t

num,file,time,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,str25,str23,float64,int32,float64,float64,float64,float64
0,AssScImHD163466_y4_1.fits,2016-05-10T08:43:22.192,3.0,23,2.80810530353,2.93709578691,0.0717107607457,0.0760826986652
1,AssScImHD163466_y4_2.fits,2016-05-10T08:43:29.513,3.0,23,2.68655532696,2.86879575208,0.0819848604541,0.0456488029924
2,AssScImHD163466_y4_3.fits,2016-05-10T08:43:36.463,3.0,25,2.64452856755,2.69076001366,0.0755942751507,0.0286223131641
3,AssScImHD163466_y4_4.fits,2016-05-10T08:43:43.448,3.0,30,2.6723065947,2.72932681468,0.0708837703989,0.249819800909
4,AssScImHD163466_y4_5.fits,2016-05-10T08:44:47.481,1.5,16,2.85770605392,2.76551370117,0.236654258462,0.0724385881085
5,AssScImHD163466_y4_6.fits,2016-05-10T08:44:52.918,1.5,14,2.78575614456,2.87539302862,0.0807985674717,0.0585594161166
6,AssScImHD163466_y4_7.fits,2016-05-10T08:44:58.453,1.5,11,2.77968098699,2.90044545098,0.0623518794852,0.0405807569436
7,AssScImHD163466_y4_8.fits,2016-05-10T08:45:04.001,1.5,13,2.66223236329,2.8031331225,0.0747038608423,0.08107409864


## Write output file

In [32]:
t.write(outputtablefile,format='fits')

## Pandas

In [33]:
df=t.to_pandas()

In [34]:
df.describe()

,num,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
count,8.00000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,3.50000,2.250000,19.375000,2.737109,2.821308,0.094335,0.081603
std,2.44949,0.801784,6.781013,0.079907,0.087665,0.057832,0.070432
min,0.00000,1.500000,11.000000,2.644529,2.690760,0.062352,0.028622
25%,1.75000,1.500000,13.750000,2.669788,2.756467,0.071504,0.044382
50%,3.50000,2.250000,19.500000,2.733118,2.835964,0.075149,0.065499
75%,5.25000,3.000000,23.500000,2.791343,2.881656,0.081095,0.077331
max,7.00000,3.000000,30.000000,2.857706,2.937096,0.236654,0.249820
